In [1]:
%tensorflow_version 1.x

import os
from os.path import exists, join, basename, splitext

project_name = 'EAST'
#git_repo_url = 'https://github.com/argman/EAST.git'
git_repo_url = 'https://github.com/renjithsasidharan/east-meeter-reading-detection.git'
#project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q $git_repo_url $project_name
  #!cd $project_name && pip install -q -r requirements.txt
  # patch for the Python 3.7
  !sed -i 's/python3-config/python3.7-config/g' EAST/lanms/Makefile
  
import sys
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

TensorFlow 1.x selected.


In [3]:
pretrained_model = 'east_icdar2015_resnet_v1_50_rbox'
if not exists(pretrained_model):
  # download the pretrained model
  pretrained_model_file_name = 'east_icdar2015_resnet_v1_50_rbox.zip'
  !gdown --id 0B3APw5BZJ67ETHNPaU9xUkVoV0U
  !unzip -qq $pretrained_model_file_name
  !mkdir /tmp/east_icdar2015_resnet_v1_50_rbox
  !mv east_icdar2015_resnet_v1_50_rbox/* /tmp/east_icdar2015_resnet_v1_50_rbox/


!unzip -qq training_data.zip

In [6]:
!python EAST/multigpu_train.py --gpu_list=0 --input_size=512 --batch_size_per_gpu=14 \
--checkpoint_path=/tmp/east_icdar2015_resnet_v1_50_rbox/ \
--text_scale=512 --training_data_path=training_data --geometry=RBOX --learning_rate=0.0001 \
--num_readers=24 --restore --save_checkpoint_steps=100 --batch_size_per_gpu=10 --max_steps=500

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0508 08:45:30.819310 140094719924096 module_wrapper.py:139] From EAST/multigpu_train.py:73: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W0508 08:45:30.819593 140094719924096 module_wrapper.py:139] From EAST/multigpu_train.py:80: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0508 08:45:30.829837 140094719924096 module_wrapper.py:139] From EAST/multigpu_train.py:88: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.


W0508 08:45:30.834159 140094719924096 module_wrapper.py:139] From EAST/multigpu_train.py:89: The nam

In [4]:
!ls /tmp/east_icdar2015_resnet_v1_50_rbox

checkpoint			      model.ckpt-49491.index
model.ckpt-49491.data-00000-of-00001  model.ckpt-49491.meta


In [5]:
!zip checkpoint.zip /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49896.data-00000-of-00001 /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49896.index /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49694.meta /tmp/east_icdar2015_resnet_v1_50_rbox/checkpoint

	zip warning: name not matched: /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49896.data-00000-of-00001
	zip warning: name not matched: /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49896.index
	zip warning: name not matched: /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49694.meta
  adding: tmp/east_icdar2015_resnet_v1_50_rbox/checkpoint (deflated 78%)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp /content/checkpoint.zip "/content/gdrive/MyDrive/training_data/meeter-reader/east"

Mounted at /content/gdrive


#### Covert checkpoint file to tensorlfow protobuff file. Protobuff file can be run on max/windows/linux

In [3]:
import tensorflow as tf
import EAST.model as model

tf.compat.v1.flags.DEFINE_string('f','','')
tf.app.flags.DEFINE_string('checkpoint_path', '/tmp/east_icdar2015_resnet_v1_50_rbox/', '')
FLAGS = tf.app.flags.FLAGS

with tf.get_default_graph().as_default():
        input_images = tf.placeholder(tf.float32, shape=[None, None, None, 3], name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)

        _, _ = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())

        with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
            ckpt_state = tf.train.get_checkpoint_state(FLAGS.checkpoint_path)
            model_path = os.path.join(FLAGS.checkpoint_path, os.path.basename(ckpt_state.model_checkpoint_path))
            print('Restore from {}'.format(model_path))
            saver.restore(sess, model_path)

            # frozen graph
            output_graph = "east_det.pb"
            output_graph_def = tf.graph_util.convert_variables_to_constants(
                sess,
                tf.get_default_graph().as_graph_def(),
                ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"])

            with tf.gfile.GFile(output_graph, "wb") as f:
                f.write(output_graph_def.SerializeToString())




Instructions for updating:
Please use `layer.__call__` method instead.
resnet_v1_50/block1 (?, ?, ?, 256)
resnet_v1_50/block2 (?, ?, ?, 512)
resnet_v1_50/block3 (?, ?, ?, 1024)
resnet_v1_50/block4 (?, ?, ?, 2048)
Shape of f_0 (?, ?, ?, 2048)
Shape of f_1 (?, ?, ?, 512)
Shape of f_2 (?, ?, ?, 256)
Shape of f_3 (?, ?, ?, 64)

Shape of h_0 (?, ?, ?, 2048), g_0 (?, ?, ?, 2048)
Shape of h_1 (?, ?, ?, 128), g_1 (?, ?, ?, 128)
Shape of h_2 (?, ?, ?, 64), g_2 (?, ?, ?, 64)
Shape of h_3 (?, ?, ?, 32), g_3 (?, ?, ?, 32)
Restore from /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49491
INFO:tensorflow:Restoring parameters from /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49491
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 306 variables.
INFO:tensorflow:Converted 306 variables to const ops.


#### Convert pb file to Tensorflow Lite model that can be run on mobile device

In [4]:
IMG_SIZE = 320

assert IMG_SIZE % 32 == 0, 'size has to be multiple of 32'

converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='east_det.pb', 
    input_arrays=['input_images'],
    output_arrays=['feature_fusion/Conv_7/Sigmoid', 'feature_fusion/concat_3'],
    input_shapes={'input_images': [1, IMG_SIZE, IMG_SIZE, 3]}
)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()
open('east_det_fp16.tflite', 'wb').write(tflite_model)


converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='east_det.pb', 
    input_arrays=['input_images'],
    output_arrays=['feature_fusion/Conv_7/Sigmoid', 'feature_fusion/concat_3'],
    input_shapes={'input_images': [1, IMG_SIZE, IMG_SIZE, 3]}
)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()
open('east_det_dr.tflite', 'wb').write(tflite_model)

24159856

#### Download **Tensorfow Lite** model file

In [5]:
from google.colab import files
files.download('/content/east_det_fp16.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>